# Perluasan Berbasis Topik

* infering topik spam tipe-3 pada dataset spam tipe-1
* menggunakan topic score (probability score topic) tertinggi

In [ ]:
import pandas as pd
import numpy as np
import json
import os
from gensim.models.ldamodel import LdaModel
from gensim.models.lsimodel import LsiModel
from gensim.models import HdpModel 
from gensim.models.nmf import Nmf
import matplotlib.pyplot as plt
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.models import Phrases
import pyLDAvis
import pyLDAvis.gensim
from random import randint
import ast
import logging

# Load Model Topik

* Dari TopicModeling.ipynb

In [ ]:
lsi_model = LsiModel.load('Data/Output/2. Topic Modeling/lsimodelmodel_26_12_3 topik_best2', mmap='r')
print(lsi_model)

# Load Data 

* token dari spam tipe-1 dan TopicBasedResult
* untuk infering topic dan mendapatkan topik score berupa probability score per topik spmam tipe-3 pada dataset spam tipe-1

In [ ]:
data = pd.read_csv('Data/Output/1. Preprocessing/amz_spam1_token_ag.csv', sep=',', encoding='latin-1')
df = pd.read_csv('Data/Output/1. Preprocessing/amz_spam323_token_ag.csv', sep=',', encoding='latin-1')

clean = data['clean'];
clean = list(map(ast.literal_eval,clean))

cleanb = df['clean'];
cleanb = list(map(ast.literal_eval,cleanb))

#https://markroxor.github.io/gensim/static/notebooks/lda_training_tips.html
bigram = Phrases(cleanb, min_count=10)
for idx in range(len(cleanb)):
    for token in bigram[cleanb[idx]]:
        if '_' in token:
            cleanb[idx].append(token)

# Pembentukan Bag of Word 

* index kata dan kemunculan kata

In [ ]:
#kamus berdasarkan data spam 23
dictionary = gensim.corpora.Dictionary(cleanb)

# Preview
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

dictionary.filter_extremes()

#bow dari data spam 1 karna yang ingin diklasifikasi review dari spam 1 
bow_corpus = [dictionary.doc2bow(doc) for doc in clean]

# *Infering* Topic Pada Unseen Document

* Unseen Doc : Spam Tipe-1

In [ ]:
# Buat list probabilitas tertinggi topik untuk semua dokumen
topic_list = []
score_list = []

for i in range(len(bow_corpus)):
    sorted_probability = sorted(lsi_model[bow_corpus[i]], key = lambda x: x[1], reverse=True)
    topic_list.append(sorted_probability[0][0])
    score_list.append(sorted_probability[0][1])

data['topic'] = topic_list
data['score'] = score_list 

print(data.tail(10))

* menyimpan data topik dan score topik 

In [ ]:
#data.to_csv('Data/Output/2. Topic Modeling/clean-topic-12-26_lsi-best2_07.csv', index=False)

* preview

In [ ]:
for index, score in sorted(lsi_model[bow_corpus[64]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lsi_model.print_topic(index)))

for i in range(len(bow_corpus)):  
    top_topics = lsi_model[bow_corpus[i]]

# Load Data II

* untuk menyatukan data yang terseleksi dari thresholding

In [ ]:
#data tambahan spam 1 
data_raw = pd.read_csv('Data/train_spam1 - juli.csv', sep=',', encoding='latin-1')
raw = data_raw['Body']

#data spam 2 3 + data tambahan dari prep postag

data_raw_s = pd.read_csv('Data/Output/1. Preprocessing/amz_end_prep_ag.csv', sep=',', encoding='latin-1')
raw_s = data_raw_s['clean']

# Set Threshold 

* threshold pada topic score untuk menyeleksi spam-tipe-1 yang memiliki kemiripan topik dengan spam tipe-3


In [ ]:
#data spam 3 

seq_sel=[]
for i in range(len(raw_s)):
    seq_sel.append(raw_s[i])

clean = data['clean']
score = data['score']

j=0

#data spam 3 ditambah data spam 1 yang terpilih berdasarkan score

for i in range(len(score)):
    if (score[i] > 0.7):
        j+=1
        seq_sel.append(raw[i])

# Simpan Hasil Perluasan Berbasis Topik

In [ ]:
#simpan hasil inference spam 1 ke berdasarkan topic modeling ke dalam DataFrame
        
import re
d=pd.DataFrame(seq_sel, columns=['r'])

#eliminasi data dengan jumlah kata dalam data kurang dari 10 kata
for i in range(len(d['r'])):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s',d['r'][i])
    text = d['r'][i].split()
    if  len(text)<10:
        print(text)
        d.drop(i, inplace=True)
    if i==11:
        d.drop(i, inplace=True)
    elif i==9:
        d.drop(i, inplace=True)

* kedalam csv dan txt file

In [ ]:
#simpan ke dalam excel 
d.to_csv('Data/Output/2. Topic Modeling/spam_seq_sel_ag3_12-26_lsi-best2_07_with structure prep.csv', index=False)

#simpan spam 3 dan tambahan dari spam 1 ke dlm txt
with open('Data/Output/5. Sequence/spam_seq_sel_ag3_12-26_lsi-best2_07_with structure prep.txt', 'w') as f:
    for item in d['r']:
        f.write("%s\n" % item)
        